In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///D:/bld/scikit-learn_1652976858669/work


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

C:\ProgramData\Anaconda3\envs\mlops_zoomcamp_homework\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\Anaconda3\envs\mlops_zoomcamp_homework\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

# Q1

In [5]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [9]:
import numpy as np
np.mean(y_pred)

16.191691679979066

# Q2

In [14]:
month = 2
year = 2021
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['y_pred'] = y_pred

In [15]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['y_pred'] = df['y_pred']

In [16]:
df_result.head()

,ride_id,y_pred
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [18]:
output_file = "./output/data.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [25]:
import os
os.path.getsize(output_file) / 1000000

19.711403

# Q3

In [ ]:
jupyter nbconvert --to script starter.ipynb